In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

from gen_ai_hub.proxy.native.openai import chat
from gen_ai_hub.proxy.native.google_vertexai.clients import GenerativeModel
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

In [2]:
import os
import tempfile
import git

IRRELEVANT_FILES = [
    "webapp/",
    "app/",
    "node_modules/",
    ".gitignore",
    "package-lock.json",
    "i18n/",
    "i18n"
    ".vscode",
    ".env",
    "eslint",
    ".eslintrc",
    ".git/",
    "__pycache__/",
    "core"
]
    
def collect_all_files(repo_url, irrelevant_files=IRRELEVANT_FILES):
    
    project_path = tempfile.mkdtemp()
    git.Repo.clone_from(repo_url, project_path)
    print(f"Cloned repository to {project_path}")
    
    relevant_files = []
    n_total=0
    n_relevant=0
    
    for root, dirs, files in os.walk(project_path):
        for file in files:
            
            abs_path = os.path.join(root, file)
            rel_path = os.path.relpath(abs_path, project_path)
                            
            n_total += 1
            
            if any(exclude in rel_path or exclude in files for exclude in irrelevant_files):
                n_relevant += 1

            relevant_files.append({
                "name": file,
                "full_path": abs_path,
                "rel_path": rel_path,
                "format": file.split(".")[-1] if "." in file else None,
                "is_relevant": not any(exclude in rel_path or exclude in files for exclude in irrelevant_files),    
            })

    print(f"Total files: {n_total}")
    print(f"Relevant files: {n_relevant}")
    
    return relevant_files



In [3]:
repo_files = collect_all_files("https://github.tools.sap/Delivery-Scale-PT/opportunity-assistant-CAP.git")
print(f"Collected {len(repo_files)} files from the repository.")

Cloned repository to /var/folders/9d/1rdr_39n49n113vgdr6vpndc0000gn/T/tmpixit8tva
Total files: 123
Relevant files: 107
Collected 123 files from the repository.


In [ ]:
def generate_cds_documentation(relevant_file_content, llm_model = "gpt-4o", temperature = 0):
    
    system_prompt = """
        You are a technical documentation assistant specialized in SAP CAP (Cloud Application Programming Model). 
        You will receive the full content of a CAP project, including all relevant .cds files, service implementations (.js), and metadata/configuration files.

        Your task is to generate complete, structured, and developer-friendly technical documentation in **Markdown** format. The documentation must be organized into the following clearly labeled sections:

        ---

        1. **Project File Structure**  
        - Provide a full hierarchical folder and file structure of the CAP project using indentation or tree-style formatting.

        2. **CAP Application Files Overview**  
        - Present a two-column table:  
            | File Name | Description |
        - Describe the purpose and role of each file in the application context.

        3. **Data Model Representation**  
        - Use a Markdown diagram or text-based representation showing:
            - All entities
            - Attributes (name, type, default, key)
            - Associations and compositions with cardinality (1:1, 1:n, etc.)
            - Navigation paths between entities

        4. **DB Schema**  
        - Cover all Tables, Views, and Types.
        - Use a table with the following columns:  
            | Name | Type (Table/View/Type) | Fields (name, key, type, default, annotations) | Annotations | Description |

        5. **CDS Definitions**  
        - For each CDS definition, present a table including:
            | Name | Projected DB Entity | Supported CRUD Operations | Fields (name, type, annotations) | Description | Annotations |

        6. **Function and Action Imports**  
        - Provide a table with the following columns:  
            | Name | Type (Function/Action) | Description | Supported Operation (GET/POST) | Parameters (name, type, default) | Return Type | Annotations | Associated Entities |

        7. **Event Handlers**  
        - For every handler in the implementation layer, document:  
            | Handler Type (on/before/after) | Operation (Create/Read/Update/Delete/Custom) | Target (Entity/Function/Action) | Implementation Description | Helper Functions Used |

        8. **Server Helper Functions**  
        - Extract and describe reusable helper functions defined in the `srv/` folder:  
            | Function Name | Location (relative file path) | Description | Parameters (name, type, default) | Return Type | Implementation Logic Summary |

        ---

        **⚠️ Output Instructions – Must Follow Without Exception**:
        - **Do not** omit or leave any section blank.
        - For **every detail requested**, include a placeholder if the value is missing:
            - `[N/A]` — Not applicable for this context.
            - `[EMPTY]` — Expected but no value was found in the input.
            - `[UNKNOWN]` — Could not be deduced or inferred confidently.
        - **Never** leave any field or table cell empty.
        - Avoid raw code unless absolutely necessary.
        - Provide clear, concise, and technical descriptions suitable for SAP CAP developers.

        The documentation must be copy-paste ready for Markdown viewers or Microsoft Word.
    """

    user_prompt = f"""
                    Here are the CAP project files and their contents:

                    {relevant_file_content}
                    """
                    
    proxy_client = get_proxy_client('gen-ai-hub')
    kwargs = dict({'model_name': llm_model})
    model = GenerativeModel(proxy_client=proxy_client, **kwargs)

    content = [{
        "role": "user",
        "parts": [{
            "text": system_prompt + user_prompt,
        }]
    }]
    # response = chat.completions.create(
    #                 model=llm_model,
    #                 temperature=temperature,
    #                 messages=[
    #                     {"role": "system", "content": system_prompt},
    #                     {"role": "user", "content": user_prompt}
    #                 ]
    #             )
    
    # return response.choices[0].message.content
    
    return model.generate_content(content).candidates[0].content.parts[0].text

In [17]:
import os

def load_file_content(file_list, only_relevant=True):
    content = ""

    for file in file_list: 
        header = (
                    f"\n\n===== FILE: {file['name']} =====\n"
                    f"Path   : {file['rel_path']}\n"
                    f"Format : {file['format']}\n"
                )
        
        if only_relevant and not file["is_relevant"]:
            file_content = "THIS FILE'S CONTENT IS IRRELEVANT FOR CAP PROJECT DOCUMENTATION"
        else:
            try:
                with open(file["full_path"], "r", encoding="utf-8") as f:
                    file_content = f.read()
            except Exception as e:
                file_content = f"⚠️ Skipping file {file.get('rel_path', 'unknown')} due to error: {e}"

        content += f"{header}\n{file_content}\n"
    
    return content

In [18]:
result = generate_cds_documentation(
            relevant_file_content = load_file_content(
                repo_files, only_relevant=True
            ),
            llm_model = "gemini-1.5-pro",
            temperature = 0.3
        )

In [12]:
print(result)

```markdown
# SAP CAP Project: opportunityassistant Documentation

---

## 1. Project File Structure

```
.
├── .eslintrc
├── .git
│   ├── config
│   ├── description
│   ├── HEAD
│   ├── hooks
│   │   ├── applypatch-msg.sample
│   │   ├── commit-msg.sample
│   │   ├── fsmonitor-watchman.sample
│   │   ├── post-update.sample
│   │   ├── pre-applypatch.sample
│   │   ├── pre-commit.sample
│   │   ├── pre-merge-commit.sample
│   │   ├── pre-push.sample
│   │   ├── pre-rebase.sample
│   │   ├── prepare-commit-msg.sample
│   │   ├── push-to-checkout.sample
│   │   └── update.sample
│   ├── index
│   ├── info
│   │   └── exclude
│   ├── logs
│   │   ├── HEAD
│   │   ├── refs
│   │   │   ├── heads
│   │   │   │   └── main
│   │   │   └── remotes
│   │   │       └── origin
│   │   │           └── HEAD
│   │   └── main
│   ├── objects
│   │   └── pack
│   │       ├── pack-4ee560a8274540e3aa55a3ee64ccc74383bf8635.idx
│   │       └── pack-4ee560a8274540e3aa55a3ee64ccc74383bf8635.pack
│   ├── pack

# SAP CAP Project: opportunityassistant Documentation

---

## 1. Project File Structure

```
.
├── .eslintrc
├── .git
│   ├── config
│   ├── description
│   ├── HEAD
│   ├── hooks
│   │   ├── applypatch-msg.sample
│   │   ├── commit-msg.sample
│   │   ├── fsmonitor-watchman.sample
│   │   ├── post-update.sample
│   │   ├── pre-applypatch.sample
│   │   ├── pre-commit.sample
│   │   ├── pre-merge-commit.sample
│   │   ├── pre-push.sample
│   │   ├── pre-rebase.sample
│   │   ├── prepare-commit-msg.sample
│   │   ├── push-to-checkout.sample
│   │   └── update.sample
│   ├── index
│   ├── info
│   │   └── exclude
│   ├── logs
│   │   ├── HEAD
│   │   ├── refs
│   │   │   ├── heads
│   │   │   │   └── main
│   │   │   └── remotes
│   │   │       └── origin
│   │   │           └── HEAD
│   │   └── main
│   ├── objects
│   │   └── pack
│   │       ├── pack-4ee560a8274540e3aa55a3ee64ccc74383bf8635.idx
│   │       └── pack-4ee560a8274540e3aa55a3ee64ccc74383bf8635.pack
│   ├── packed-refs
│   └── refs
│       ├── heads
│       │   └── main
│       └── remotes
│           └── origin
│               └── HEAD
├── .gitignore
├── .vscode
│   ├── extensions.json
│   ├── launch.json
│   └── tasks.json
├── README.md
├── app
│   ├── opportunityassistantcockpit
│   │   ├── annotations.cds
│   │   ├── css
│   │   │   └── style.css
│   │   ├── controller
│   │   │   ├── App.controller.js
│   │   │   ├── Assistant.controller.js
│   │   │   ├── BaseController.js
│   │   │   ├── Files.controller.js
│   │   │   ├── Home.controller.js
│   │   │   ├── Settings.controller.js
│   │   │   └── Sources.controller.js
│   │   ├── control
│   │   │   ├── D3Chart.js
│   │   │   ├── D3ColumnChart.js
│   │   │   ├── D3ComparisonChart.js
│   │   │   └── D3PieChart.js
│   │   ├── i18n
│   │   │   ├── i18n.properties
│   │   │   └── terminologies
│   │   │       ├── sports
│   │   │       │   ├── i18n.terminologies.sports_de.properties
│   │   │       │   └── i18n.terminologies.sports_en.properties
│   │   │       └── travel
│   │   │           ├── i18n.terminologies.travel_de.properties
│   │   │           └── i18n.terminologies.travel_en.properties
│   │   ├── lib
│   │   │   ├── axios.min.js
│   │   │   ├── opencv.js
│   │   │   └── tfjs.js
│   │   ├── model
│   │   │   ├── formatter.js
│   │   │   ├── models.js
│   │   │   └── sideContent.json
│   │   ├── package-lock.json
│   │   ├── package.json
│   │   ├── README.md
│   │   ├── reuse
│   │   │   ├── appvar1
│   │   │   │   ├── i18n
│   │   │   │   │   ├── i18n_de.properties
│   │   │   │   │   ├── i18n_en.properties
│   │   │   │   │   └── terminologies
│   │   │   │   │       ├── sports
│   │   │   │   │       │   ├── i18n.terminologies.sports_de.properties
│   │   │   │   │       │   └── i18n.terminologies.sports_en.properties
│   │   │   │   │       └── travel
│   │   │   │   │           ├── i18n.terminologies.travel_de.properties
│   │   │   │   │           └── i18n.terminologies.travel_en.properties
│   │   │   │   └── images
│   │   │   │       ├── sports
│   │   │   │       │   ├── header_sports.jpg
│   │   │   │       │   └── header_sports_small.jpg
│   │   │   │       └── travel
│   │   │   │           ├── header_travel.jpg
│   │   │   │           └── header_travel_small.jpg
│   │   │   └── appvar2
│   │   │       ├── i18n
│   │   │       │   ├── i18n_de.properties
│   │   │       │   ├── i18n_en.properties
│   │   │       │   └── terminologies
│   │   │       │       └── travel
│   │   │       │           ├── i18n.terminologies.travel_de.properties
│   │   │       │           └── i18n.terminologies.travel_en.properties
│   │   │       └── images
│   │   │           └── travel
│   │   │               ├── header_travel.jpg
│   │   │               └── header_travel_small.jpg
│   │   ├── test
│   │   │   ├── integration
│   │   │   │   ├── AllJourneys.js
│   │   │   │   ├── arrangements
│   │   │   │   │   └── Startup.js
│   │   │   │   ├── NavigationJourney.js
│   │   │   │   ├── opaTests.qunit.html
│   │   │   │   ├── opaTests.qunit.js
│   │   │   │   └── pages
│   │   │   │       ├── App.js
│   │   │   │       ├── Home.js
│   │   │   │       ├── Settings.js
│   │   │   │       └── Statistics.js
│   │   │   └── unit
│   │   │       ├── AllTests.js
│   │   │       ├── model
│   │   │       │   ├── formatter.js
│   │   │       │   └── models.js
│   │   │       ├── unitTests.qunit.html
│   │   │       └── unitTests.qunit.js
│   │   ├── ui5-deploy.yaml
│   │   ├── ui5.yaml
│   │   ├── util
│   │   │   └── ServiceUtil.js
│   │   ├── view
│   │   │   ├── App.view.xml
│   │   │   ├── Assistant.view.xml
│   │   │   ├── Files.view.xml
│   │   │   ├── Home.view.xml
│   │   │   ├── Settings.view.xml
│   │   │   └── Sources.view.xml
│   │   ├── webapp
│   │   │   ├── Component.js
│   │   │   ├── css
│   │   │   │   └── style.css
│   │   │   ├── index.html
│   │   │   ├── i18n
│   │   │   │   ├── i18n.properties
│   │   │   │   └── terminologies
│   │   │   │       ├── sports
│   │   │   │       │   ├── i18n.terminologies.sports_de.properties
│   │   │   │       │   └── i18n.terminologies.sports_en.properties
│   │   │   │       └── travel
│   │   │   │           ├── i18n.terminologies.travel_de.properties
│   │   │           └── i18n.terminologies.travel_en.properties
│   │   │   ├── lib
│   │   │   │   ├── axios.min.js
│   │   │   │   ├── opencv.js
│   │   │   │   └── tfjs.js
│   │   │   ├── manifest.json
│   │   │   ├── model
│   │   │   │   ├── formatter.js
│   │   │   │   ├── models.js
│   │   │   │   └── sideContent.json
│   │   │   ├── reuse
│   │   │   │   ├── appvar1
│   │   │   │   │   ├── i18n
│   │   │   │   │   │   ├── i18n_de.properties
│   │   │   │   │   │   ├── i18n_en.properties
│   │   │   │   │   │   └── terminologies
│   │   │   │   │   │       ├── sports
│   │   │   │   │   │       │   ├── i18n.terminologies.sports_de.properties
│   │   │   │   │   │       │   └── i18n.terminologies.sports_en.properties
│   │   │   │   │   │       └── travel
│   │   │   │   │   │           ├── i18n.terminologies.travel_de.properties
│   │   │           └── i18n.terminologies.travel_en.properties
│   │   │   │   └── images
│   │   │   │       ├── sports
│   │   │   │       │   ├── header_sports.jpg
│   │   │   │       │   └── header_sports_small.jpg
│   │   │   │       └── travel
│   │   │   │           ├── header_travel.jpg
│   │   │   │           └── header_travel_small.jpg
│   │   │   │   └── appvar2
│   │   │   │       ├── i18n
│   │   │   │       │   ├── i18n_de.properties
│   │   │   │       │   ├── i18n_en.properties
│   │   │   │       │   └── terminologies
│   │   │   │       │       └── travel
│   │   │   │       │           ├── i18n.terminologies.travel_de.properties
│   │   │   │           └── i18n.terminologies.travel_en.properties
│   │   │   │       └── images
│   │   │   │           └── travel
│   │   │   │               ├── header_travel.jpg
│   │   │               └── header_travel_small.jpg
│   │   │   ├── test
│   │   │   │   ├── integration
│   │   │   │   │   ├── AllJourneys.js
│   │   │   │   │   ├── arrangements
│   │   │   │   │   │   └── Startup.js
│   │   │   │   │   ├── NavigationJourney.js
│   │   │   │   │   ├── opaTests.qunit.html
│   │   │   │   │   ├── opaTests.qunit.js
│   │   │   │   │   └── pages
│   │   │   │   │       ├── App.js
│   │   │   │   │       ├── Home.js
│   │   │   │   │       ├── Settings.js
│   │   │   │       └── Statistics.js
│   │   │   │   └── unit
│   │   │   │       ├── AllTests.js
│   │   │   │       ├── model
│   │   │   │       │   ├── formatter.js
│   │   │   │       │   └── models.js
│   │   │   │       ├── unitTests.qunit.html
│   │   │       └── unitTests.qunit.js
│   │   │   ├── util
│   │   │   │   └── ServiceUtil.js
│   │   │   └── view
│   │   │       ├── App.view.xml
│   │   │       ├── Assistant.view.xml
│   │   │       ├── Files.view.xml
│   │   │       ├── Home.view.xml
│   │   │       ├── Settings.view.xml
│   │   │       └── Sources.view.xml
│   │   └── xs-app.json
│   └── services.cds
├── core
├── db
│   ├── schema.cds
│   ├── src
│   │   └── .hdiconfig
│   └── undeploy.json
├── mta.yaml
├── package-lock.json
├── package.json
├── srv
│   ├── AICoreUtil.js
│   ├── AICoreUtilChatbot.js
│   ├── api.cds
│   ├── api.js
│   ├── bucketUtil.js
│   ├── CFUtil.js
│   ├── server.js
│   ├── _i18n
│   │   └── i18n.properties
│   └── util
│       ├── AxiosUtil.js
│       ├── DataServiceUtil.js
│       └── sharepoint.js
└── xs-security.json

```

## 2. CAP Application Files Overview

| File Name | Description |
|---|---|
| `app/services.cds` | [IRRELEVANT FOR CAP PROJECT DOCUMENTATION] |
| `app/opportunityassistantcockpit/*` | UI module for the application. Contains UI5 application code, views, controllers, etc. |
| `db/schema.cds` | Defines the data model (entities, types) for the application. |
| `db/undeploy.json` |  Configuration file for undeploying database artifacts. |
| `db/src/.hdiconfig` | HANA Deployment Infrastructure Configuration file. |
| `srv/api.cds` | Defines the service definitions, exposing data models and custom handlers. |
| `srv/server.js` | Main server file, handles bootstrapping and other server-side logic including Websocket connections. |
| `srv/api.js` | Service implementation file containing the logic for handling requests.  |
| `srv/AICoreUtil.js` | Utility functions for interaction with AI Core service. |
| `srv/AICoreUtilChatbot.js` |  Utility functions specifically for chatbot interactions with AI Core. |
| `srv/CFUtil.js` | Utility functions related to Cloud Foundry environment and MS Graph interaction. |
| `srv/bucketUtil.js` | Utility functions for AWS S3 Bucket operations (upload, download, delete, etc.). |
| `srv/util/AxiosUtil.js` | Reusable Axios utility for HTTP requests with token handling. |
| `srv/util/DataServiceUtil.js` | Utility functions for interacting with a data service, likely using AxiosUtil. |
| `srv/util/sharepoint.js` | Utility functions for SharePoint integration using MSAL library. |



## 3. Data Model Representation

```
entity EntityObject {
  key ID          : UUID managed;
  external_id : String(100);
  description : String;
  metadata    : String(5000);
  status      : String enum { INITIAL, FILES_UPLOADED, FILES_STORED_IN_VECTOR_DB } default 'INITIAL';
  files       : Composition of many EntityObjectFile on files.entityObject = $self;
}

entity EntityObjectFile {
  key ID          : UUID;
  filename     : String;
  location     : String;
  size         : String;
  type         : String;
  comments     : String;
  entityObject : Association to one EntityObject; 
}
```

- **EntityObject** and **EntityObjectFile** have a composition relationship (1:n).
- `EntityObject` has a `files` attribute that allows navigation to multiple associated `EntityObjectFile` entities.
- `EntityObjectFile` can navigate back to its parent `EntityObject` via the `entityObject` attribute.

## 4. DB Schema

| Name | Type (Table/View/Type) | Fields (name, key, type, default, annotations) | Annotations | Description |
|---|---|---|---|---|
| EntityObject | Table | ID (key, UUID, managed), external_id (String(100)), description (String), metadata (String(5000)), status (String, default 'INITIAL', enum: INITIAL, FILES_UPLOADED, FILES_STORED_IN_VECTOR_DB) | `managed`, `cuid` | Stores information about entity objects. |
| EntityObjectFile | Table | ID (key, UUID), filename (String), location (String), size (String), type (String), comments (String), entityObject_ID (UUID) | `cuid` | Stores files related to entity objects. |
| STATUS | Type | `[N/A]` |  | Enum type defining the status of an entity object. |


## 5. CDS Definitions

| Name | Projected DB Entity | Supported CRUD Operations | Fields (name, type, annotations) | Description | Annotations |
|---|---|---|---|---|---|
| APIService.EntityObject | db.EntityObject |  CRUD | ID, external_id, description, metadata, status, files | Service entity for EntityObject. | `[N/A]` |
| APIService.EntityObjectFile | db.EntityObjectFile | CRUD | ID, filename, location, size, type, comments, entityObject | Service entity for EntityObjectFile. | `[N/A]` |


## 6. Function and Action Imports

| Name | Type (Function/Action) | Description | Supported Operation (GET/POST) | Parameters (name, type, default) | Return Type | Annotations | Associated Entities |
|---|---|---|---|---|---|---|---|
| getAdminOptions | Function | Retrieves admin options. | GET |  | String |  | AdminPreference |
| setAdminOptions | Action | Sets admin options. | POST | defaultApprover (String) | [N/A] |  | AdminPreference |
| chatMemory | Action | Queries the chatbot with memory context. | POST | query (String) | String | | [N/A] |
| chatHistoryByUserId | Action | Retrieves chat history for a specific user. | GET | | String |  | [N/A] |
| isChatbotEnabled | Function | Checks if the chatbot is enabled. | GET | | Boolean | | [N/A] |
| getSharepointConnectUrl | Function | Retrieves the SharePoint connect URL. | GET | | String |  | [N/A] |
| getAzureToken | Function | Retrieves the Azure token. | GET | | String | | [N/A] |
| getBidCases | Function | Retrieves bid cases. | GET | top (Integer), skip (Integer) | array of String |  | [N/A] |
| getBidCasesWithDeliverables | Function | Retrieves bid cases with deliverables. | GET | top (Integer), skip (Integer), caseId (String) | String |  | [N/A] |
| getStats | Function | Retrieves statistics. | GET | | String |  | [N/A] |
| isAzureTokenValid | Function | Checks if the Azure token is valid. | GET | | String | | [N/A] |
| getFilesByCaseId | Function | Retrieves files by case ID. | GET | caseId (String) | String |  | EntityObjectFile |
| getAllFiles | Function | Retrieves all files. | GET | | String | | EntityObjectFile |
| searchFilesByExternalIdOrFileName | Function | Searches files by external ID or file name. | GET | search (String) | String | | EntityObjectFile |
| deleteFilesByCaseId | Action | Deletes files by case ID. | POST | caseId (String) | String | | EntityObjectFile |
| refreshChat | Action | Refreshes the chat. | POST | model (String) | String |  | [N/A] |
| loadBidCaseDeliverablesToVectorDB | Function | Loads bid case deliverables to vector DB. | POST | caseId (String) | String | | [N/A] |


## 7. Event Handlers

| Handler Type (on/before/after) | Operation (Create/Read/Update/Delete/Custom) | Target (Entity/Function/Action) | Implementation Description | Helper Functions Used |
|---|---|---|---|---|
| `on` | `getBidCases` | `getBidCases` | Retrieves bid cases from a data service using `dataServiceUtil.getBidCases`. | `dataServiceUtil.getBidCases` |
| `on` | `getBidCasesWithDeliverables` | `getBidCasesWithDeliverables` | Retrieves bid cases with deliverables and their associated files. It also checks for ongoing ingestion processes. | `dataServiceUtil.getBidCasesWithDeliverables`, `dataServiceUtil.getFilesByCaseId`, `checkLock` |
| `on` | `loadBidCaseDeliverablesToVectorDB` | `loadBidCaseDeliverablesToVectorDB` | Loads bid case deliverables to a vector database. Manages a lock to prevent concurrent ingestion.  | `dataServiceUtil.loadBidCaseDeliverablesToVectorDB`, `addLock`, `dropLock` |
| `on` | `getFilesByCaseId` | `getFilesByCaseId` | Retrieves files associated with a specific case ID using `dataServiceUtil.getFilesByCaseId`. | `dataServiceUtil.getFilesByCaseId` |
| `on` | `deleteFilesByCaseId` | `deleteFilesByCaseId` | Deletes files associated with a given case ID. | `dataServiceUtil.deleteFilesByCaseId` |
| `on` | `getAllFiles` | `getAllFiles` | Retrieves all files using `dataServiceUtil.getAllFiles`. | `dataServiceUtil.getAllFiles` |
| `on` | `refreshChat` | `refreshChat` | Refreshes the chat memory. | `dataServiceUtil.refreshChat` |
| `on` | `getStats` | `getStats` | Retrieves statistics about total files and bid cases. | `dataServiceUtil.getAllFiles` |
| `on` | `getSharepointConnectUrl` | `getSharepointConnectUrl` | Retrieves the SharePoint connect URL using  `getSharepointConnectUrl` utility. | `getContext`, `getSharepointConnectUrl` |
| `on` | `getAzureToken` | `getAzureToken` | Handles retrieval and storage of Azure tokens. | `getContext`, `connectSharepoint`, `AxiosUtil.checkTokenExpired`, `LOG.log` |
| `on` | `isAzureTokenValid` | `isAzureTokenValid` | Checks Azure token validity and expiration. | `AxiosUtil.checkTokenExpired` |

| `on` | `updatePreference` | `updatePreference` | Updates user preferences. |  |
| `on` | `userInfo` | `userInfo` | Retrieves user information. | `checkNull` |
| `on` | `searchFilesByExternalIdOrFileName` | `searchFilesByExternalIdOrFileName` | Searches files by external ID or file name. | `dataServiceUtil.searchFilesByExternalIdOrFileName` |
| `on` | `getAdminOptions` | `getAdminOptions` | Retrieves admin options, initializes them if they don't exist. |  |
| `on` | `setAdminOptions` | `setAdminOptions` | Sets admin options. |  |
| `on` | `chatMemory` | `chatMemory` | Sends a query to the chatbot with memory context. | `dataServiceUtil.chatMemory` |
| `on` | `chatHistoryByUserId` | `chatHistoryByUserId` | Retrieves the chat history for a specific user. | `dataServiceUtil.chatHistoryByUserId` |
| `on` | `isChatbotEnabled` | `isChatbotEnabled` | Checks if the chatbot feature is enabled. |  |



## 8. Server Helper Functions

| Function Name | Location (relative file path) | Description | Parameters (name, type, default) | Return Type | Implementation Logic Summary |
|---|---|---|---|---|---|
| `emitMessageToClients` | `srv/server.js` | Sends a message to all connected WebSocket clients. | `message` (any) | `void` | Iterates through connected WebSocket clients and sends the given message to each client whose `readyState` is 1 (OPEN). |

| `getToken` | `srv/AICoreUtilChatbot.js` | Retrieves an access token for AI Core. |  | `Promise<string>` | Makes a request to the AI Core authentication URL to get an access token using client credentials.  |
| `chat` | `srv/AICoreUtilChatbot.js` | Sends a chat query to AI Core. | `sQuestion` (string) | `Promise<object>` | Sends a POST request to the AI Core chat endpoint with the provided query. |
| `chatWithMemory` | `srv/AICoreUtilChatbot.js` | Sends a chat query to AI Core with memory context. | `sQuestion` (string), `sUserId` (string) | `Promise<object>` | Similar to `chat` but includes a `user_id` for memory context. |
| `getStatus` | `srv/AICoreUtilChatbot.js` | Retrieves the status of AI Core. |  | `Promise<object>` | Sends a GET request to the AI Core status endpoint. |
| `getRefresh` | `srv/AICoreUtilChatbot.js` |  Triggers a refresh in AI Core. | | `Promise<object>` | Sends a GET request to the AI Core refresh endpoint. |
| `getTokenCF` | `srv/CFUtil.js` | Retrieves a token for Cloud Foundry. | | `Promise<string>` | Retrieves a token from the Cloud Foundry API using provided credentials. |
| `sendEmail` | `srv/CFUtil.js` | Sends an email using Microsoft Graph API. | `sEmail` (string), `sContent` (string) | `Promise<void>` | Sends an email using Microsoft Graph API with specified subject, body and recipients. |
| `getMSGraphToken` | `srv/CFUtil.js` | Retrieves a token for Microsoft Graph API. |  | `Promise<string>` | Retrieves a token from the configured Microsoft Graph API destination.  |
| various functions | `srv/bucketUtil.js` | Provides various utility functions for interacting with AWS S3 buckets. | See implementation for details. | Varies | Includes functions like `uploadFile`, `downloadFile`, `deleteFile`, `deleteBucket`, `listObjects`, `listBuckets`, `bucketExists`, `createBucket`, `listObjectsWithBase64Content`, `deleteObjectsWithMetadata`, `listAllObjects`, `getObjectContentByKey`, `listAllMyObjects`. |
| `getToken` | `srv/AICoreUtil.js` | Retrieves a token. |  | `Promise<string>` | Makes a request to retrieve a token using client ID and secret. |
| `getStatus` | `srv/AICoreUtil.js` | Gets the status. |  `token` (string, optional) | `Promise<object>` | Gets the status, optionally using a provided token or fetching a new one. |
| `getModelStatus` | `srv/AICoreUtil.js` | Gets the model status. |  `req` (object), `token` (string, optional) | `Promise<object>` | Gets the model status for a given entity, optionally using a provided token. |
| `createExecution` | `srv/AICoreUtil.js` | Creates an execution. |  | `Promise<object>` | Creates an execution with a given configuration ID. |
| `predict` | `srv/AICoreUtil.js` | Makes a prediction. |  `req` (object), `token` (string, optional) | `Promise<object>` | Makes a prediction using provided content, entity, and percentile. |
| `refreshModelServer` | `srv/AICoreUtil.js` | Refreshes the model server. |  | `Promise<object>` | Refreshes the model server. |
| `getContext` | `srv/api.js` | Extracts context information from the request. | `context` (object) | `object` | Extracts relevant details from the request context, including user ID, protocol, host, and data. |
| `checkNull` | `srv/api.js` | Checks if a value is null. | `value` (any) | `any` | Returns the value if not null, otherwise returns null. |
| `checkLock` | `srv/api.js` | Checks if a lock exists for a given case ID. | `caseId` (string) | `boolean` |  Checks if a lock entry exists in `ingestLocks` for the given `caseId`.  |
| `addLock` | `srv/api.js` | Adds a lock for a given case ID. | `caseId` (string) | `void` | Adds a lock entry to `ingestLocks` for the given `caseId`. |
| `dropLock` | `srv/api.js` | Removes a lock for a given case ID. | `caseId` (string) | `void` | Removes a lock entry from `ingestLocks` if it exists for the given `caseId`.  |
| various methods | `srv/util/AxiosUtil.js` | Provides utility methods for making HTTP requests with Axios, including token handling.  | See implementation for details | Varies | Includes methods for getting, posting, putting, deleting, refreshing tokens, and handling errors. |
| various methods | `srv/util/DataServiceUtil.js` | Provides utility functions for data service interaction. | See implementation for details | Varies | Methods for fetching and managing bid cases, files, and chat interactions with an external API using AxiosUtil. |
| `getSharepointConfig` | `srv/util/sharepoint.js` |  Retrieves SharePoint configuration. | `context` (object) | object | Returns an object containing SharePoint configuration details including MSAL configuration, redirect URIs, and Graph API endpoint. |

